# <font color=#8B4513 size=100 face="標楷體"> 謝榮哥,帆哥 </font>

# <font color=#8B4513 size=100 face="標楷體"> 資策會網路有夠爛 </font>

# <font color=#8B4513 size=100 face="標楷體"> 信用卡FB粉專_老狐狸的信用卡日記 </font>

In [1]:
import pandas as pd
import re, time, requests, datetime, gc, os, sys
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
def 垃圾視窗NotNow():
    try:
        driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
    except:
#         print(sys.exc_info())
        time.sleep(0.1)

In [3]:
# 截取塗鴉墻上貼文的連結
def fuck_Wall_PostLink(i):
    Link = 'https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0]
    return Link
#截取塗鴉墻上貼文的發佈時間
def fuck_Wall_PostTime(i):
    try:
        Time = i.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = '沒發文時間阿肏你的'
    return Time
# 截取塗鴉墻上貼文的留言數
def fuck_Wall_PostComments(i):
    try:
        Comments = i.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in Comments:
            Comments = int(float(Comments.split('K')[0])*1000)
        else:
            Comments = int(Comments)
    except:
        Comments = 0
    return Comments
def CarwlList(urls, n, Posts):
    CheckList = pd.DataFrame(columns = ['Link','Time','Comments']) 

    driver.get(url)        
    for i in range(n):
        try:
            time.sleep(1)
            driver.find_element_by_css_selector('a.pam.uiBoxLightblue.uiMorePagerPrimary').click() # 加載下一頁
        except:
            time.sleep(1)
        #會跳出強迫登入的畫面喔肏
        垃圾視窗NotNow()
        soup = BeautifulSoup(driver.page_source)
        for i in soup.find_all('div', {'class':'_5pcr userContentWrapper'}):
            CheckList = pd.concat([CheckList, pd.DataFrame(data = [{'Link':fuck_Wall_PostLink(i),
                                                                    'Time':fuck_Wall_PostTime(i),
                                                                    'Comments':fuck_Wall_PostComments(i)}],
                                                           columns = ['Link','Time','Comments'])],
                                   ignore_index = True)
    #襙有重複問題為啥啊?
    CarwlList = pd.merge(left = CheckList,
                         right = Posts.loc[:,['Link','Comments']],
                         how='left',
                         on='Link',
                         suffixes=('_c', '_p'),
                         indicator=True)
    CarwlList = pd.concat([CarwlList.loc[CarwlList._merge == 'left_only', :], # 新貼文
                           CarwlList.loc[CarwlList.Comments_c > CarwlList.Comments_p, :]], # 舊貼文但有新留言
                          ignore_index=True)
    CarwlList = CarwlList.drop_duplicates(subset = 'Link',
                                          keep= 'first',
                                          inplace = False)
    return CarwlList

# <font color=#8B4513 size=100 face="標楷體"> 彈窗 </font>

In [4]:
# 關閉彈窗
def ClosePopup():
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    垃圾視窗NotNow()
    driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
    time.sleep(1)
    垃圾視窗NotNow()
    time.sleep(0.5)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    垃圾視窗NotNow()
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    垃圾視窗NotNow()



# <font color=#8B4513 size=100 face="標楷體"> 展開留言 </font>


In [5]:
def open_comment():
    time.sleep(1)
    k = len(driver.find_elements_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]'))
    if k !=0:
        #這邊似乎FB不是很穩,往下拉兩次
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        垃圾視窗NotNow()
        try:
            driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
        except:
            print('fuckyou')
    time.sleep(1)


def ClickOldest():
    time.sleep(1)
    driver.find_element_by_xpath('//a[@data-testid="UFI2ViewOptionsSelector/link"]').click()
    time.sleep(1)
    try:
        driver.find_element_by_partial_link_text('Comments shown in chronological order with the newest comments at the top.').click()
    except:
        try:
            driver.find_element_by_partial_link_text('New comments and those with new replies appear at the top.').click()
        except:
            print('Plz, Check this post arragne type!')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    垃圾視窗NotNow()
    time.sleep(1)
def MoreComments():
    k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 300):
        print('展開夠多留言： ' + str(l) + ' 次')
        try:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            垃圾視窗NotNow()
            driver.find_element_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]').click()
            time.sleep(1)
            k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))
        except:
            time.sleep(0.1)
        finally:
            l += 1
    time.sleep(1)
# 偵測是否有「更多回覆」（第二層），若有則點擊
def MoreReplies():
    k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 300):
        l += 1
        print('加載： ' + str(l) + ' 次')
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.find_element_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]').click()
        time.sleep(0.5)
        k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))
    time.sleep(1)
# 偵測是否有「展開留言」（第二層留言內容），若有則點擊
def SeeMore():
    k = len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 1000):
        l += 1
        print('Click See More ： ' + str(l) + ' times.')
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.find_element_by_xpath('//a[@class="_5v47 fss"]').click()
        time.sleep(0.5)
        k = len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))
    time.sleep(1)
def PostExpand():
#     ClosePopup()
#     ClickOldest() # 展開留言會有無法加載更多留言的問題，暫時先不使用
    open_comment()
    MoreComments()
    MoreReplies()
    SeeMore()

# 關閉彈窗
def ClosePopup():
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    垃圾視窗NotNow()
    driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
    time.sleep(1)
    垃圾視窗NotNow()
    time.sleep(0.5)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    垃圾視窗NotNow()
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    垃圾視窗NotNow()


# <font color=#8B4513 size=100 face="標楷體"> 貼文時間與內容 </font>

In [6]:
# 貼文留言數
def Post_Name(userContent):
    return userContent.find('img').attrs['aria-label']

# 貼文ID?
def Post_ID(userContent):
    return userContent.find('a', {'class':'_5pb8 o_c3pynyi2g _8o _8s lfloat _ohe'}).attrs['href'].split('/?ref')[0].split('https://www.facebook.com/')[1]
# 貼文留言數
def Post_Comments(userContent):
    try:
        CommentCounts = userContent.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in CommentCounts:
            CommentCounts = int(float(CommentCounts.split('K')[0])*1000)
        else:
            CommentCounts = int(CommentCounts)
    except:
        CommentCounts = 0 
    return CommentCounts
# 貼文時間
def Post_Time(userContent):
    try:
        Time = userContent.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = "Error, Please check this post's condiction!" 
    return Time

# 貼文內容
def Post_Content(userContent):
    try:
        Content = userContent.find('div', {'class':'_5pbx userContent _3576'}).text
    except:
        Content = "There's No Text Content!"
    return Content
# 文章內容與互動摘要
def PostInfo(soup):
    # 貼文區
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    PostContent = pd.DataFrame(data = [{'Name':Post_Name(userContent),
                                        'ID':Post_ID(userContent),
                                        'Time':Post_Time(userContent),
                                        'Content':Post_Content(userContent),
                                        'Comments':Post_Comments(userContent),
                                        'Link':i}],
                            columns = ['Name', 'ID', 'Time', 'Content', 'Comments', 'Link'])
    return PostContent


        

# <font color=#8B4513 size=100 face="標楷體"> 留言內容 </font>

In [7]:
def Comment_Content(element):
    try:
        Content = element.find('span', {'dir':'ltr'}).text
    except:
        Content = 'img'
    return Content
# def CommentsInfo(soup):  
#     PostComments = pd.DataFrame()
#     userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
#     try:
#         for i in userContent.select('ul._7a9a > li'):
#             # 先抓留言並放在Comment
#             Comment = pd.DataFrame(data=[{'ID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
#                                           'Name':i.find('img').attrs['alt'],
#                                           'Time':datetime.datetime.strptime(i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
#                                           'Content':Comment_Content(i),
#                                           'RepID':userContent.find('div', {'class':'_5pcp _5lel _2jyu _232_'}).attrs['id'].split(';')[0].split('feed_subtitle_')[-1],
#                                           'RepName':userContent.find('img').attrs['aria-label'],
#                                           'Link':driver.current_url}],
#                                    columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
#             PostComments = pd.concat([PostComments, Comment], ignore_index=True)
#             # 留言的留言
#             for j in i.findAll('div', {'data-testid':'UFI2Comment/root_depth_1'}):
#                 Comment = pd.DataFrame(data=[{'ID':j.find('a', {'class':' _3mf5 _3mg1'}).attrs['data-hovercard'].split('id=',2)[1],
#                                               'Name':j.find('img').attrs['alt'],
#                                               'Time':datetime.datetime.strptime(j.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
#                                               'Content':Comment_Content(j),
#                                               'RepID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
#                                               'RepName':i.find('img').attrs['alt'],
#                                               'Link':driver.current_url}],
#                                        columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
#                 PostComments = pd.concat([PostComments, Comment], ignore_index=True)
#     except:
#         print('抓完囉!')
#     return PostComments
def 操你的留言(soup):  
    PostComments = pd.DataFrame()
    留言的留言 = soup.find('div', {'class':'_5pcr userContentWrapper'})
    try:
        for 靠 in 留言的留言:
            PostComments = pd.DataFrame(data=[{'Content': ','.join(幹.text for 幹 in 靠.select('ul[class="_7a9a"] span[class="_3l3x"]')),
                                         'Time':'隨便啦',
                                         'ID':'你猜',
                                         'RepID':'不重要',
                                         'RepName':'嘿嘿',
                                         'Name':'SKT',
                                         'Link':i
                                         }],                         
                                   columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])





#             PostComments = pd.concat([PostComments, Comment], ignore_index=True)

    except:
         print('我是遊覽器我是廢物')
    return PostComments

## <table><tr><td bgcolor=orange> 以上是定義區</td></tr></table>

# <font color=#FF0000 size=100 face="標楷體"> 分隔線--------------------------------- </font>

## <table><tr><td bgcolor=orange> 以下是執行區</td></tr></table>

# <font color=#8B4513 size=100 face="標楷體"> excel_post_Comments </font>

In [8]:
try:
    Posts = pd.read_excel('C:\\Users\\Big data\\Desktop\\class\\web_crawler\\FB\\F1B.xls')
except:
    Posts = pd.DataFrame(columns=['Name', 'ID', 'Time', 'Content', 'Comments', 'Link'])
try:
    Comments = pd.read_pickle('C:\\Users\\Big data\\Desktop\\class\\web_crawler\\FB\\FB.xlsx')
except:
    Comments = pd.DataFrame(columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])

print(Posts.shape)
print(Comments.shape)
# #可放多個連結


(0, 6)
(0, 7)


#  <font color=#8B4513 size=100 face="標楷體"> 開啟webdriver </font>

In [9]:
url='https://www.facebook.com/SourSoul.cc/?ref=br_rs'
driver = webdriver.Chrome('C:\\Users\\Big data\\Desktop\\class\\web_crawler\\chromedriver\\chromedriver.exe')
driver.get('https://www.facebook.com/')
time.sleep(1)
driver.find_element_by_partial_link_text('English').click()


In [10]:
CarwlList你妹 = CarwlList(urls=url, n=100, Posts = Posts)#n是下拉次數,次數愈多能加載的文章愈多
CarwlList你妹

,Link,Time,Comments_c,Comments_p,_merge
0,https://www.facebook.com/SourSoul.cc/posts/270...,2019-10-17 17:30,7,NaN,left_only
1,https://www.facebook.com/SourSoul.cc/posts/275...,2019-11-05 06:10,10,NaN,left_only
2,https://www.facebook.com/SourSoul.cc/posts/274...,2019-11-04 06:00,7,NaN,left_only
3,https://www.facebook.com/SourSoul.cc/posts/274...,2019-11-04 04:00,9,NaN,left_only
4,https://www.facebook.com/SourSoul.cc/photos/a....,2019-11-03 21:00,9,NaN,left_only
5,https://www.facebook.com/SourSoul.cc/posts/274...,2019-11-01 22:00,2,NaN,left_only
12,https://www.facebook.com/SourSoul.cc/posts/274...,2019-10-31 23:10,9,NaN,left_only
13,https://www.facebook.com/SourSoul.cc/posts/274...,2019-10-31 18:16,22,NaN,left_only
14,https://www.facebook.com/SourSoul.cc/photos/a....,2019-10-30 04:00,12,NaN,left_only
15,https://www.facebook.com/SourSoul.cc/photos/a....,2019-10-28 07:16,5,NaN,left_only


In [15]:
aaa=CarwlList你妹[400:500]
aaa


,Link,Time,Comments_c,Comments_p,_merge
10373,https://www.facebook.com/SourSoul.cc/posts/198...,2018-08-05 23:14,1,NaN,left_only
10374,https://www.facebook.com/SourSoul.cc/posts/198...,2018-08-03 20:00,5,NaN,left_only
10375,https://www.facebook.com/SourSoul.cc/photos/a....,2018-08-02 03:30,32,NaN,left_only
10376,https://www.facebook.com/SourSoul.cc/posts/197...,2018-08-01 20:36,0,NaN,left_only
10377,https://www.facebook.com/SourSoul.cc/posts/197...,2018-08-01 18:44,5,NaN,left_only
10784,https://www.facebook.com/SourSoul.cc/posts/196...,2018-07-27 01:25,2,NaN,left_only
10785,https://www.facebook.com/SourSoul.cc/posts/196...,2018-07-23 21:02,1,NaN,left_only
10786,https://www.facebook.com/SourSoul.cc/posts/195...,2018-07-21 08:05,2,NaN,left_only
10787,https://www.facebook.com/SourSoul.cc/posts/163...,2017-10-31 17:41,0,NaN,left_only
10788,https://www.facebook.com/SourSoul.cc/posts/159...,2017-09-30 08:53,0,NaN,left_only


# <font color=#8B4513 size=100 face="標楷體"> 爬資料(雙表,可擴展,目前暫定把留言與連言回復合併為一表) </font>

In [16]:
nComments = pd.DataFrame(columns=['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
nPost = pd.DataFrame(columns=[ 'Name','ID', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
for i in CarwlList你妹.Link:
    print('正在抓: ' + i)
    driver.get(i)
    try:
        PostExpand()
        print('完成!')
        try:
            soup = BeautifulSoup(driver.page_source)
            time.sleep(1)
            PostContent = PostInfo(soup)
            nPost = pd.concat([nPost, PostContent],sort=True, ignore_index=True)
            PostComments = 操你的留言(soup)
            nComments = pd.concat([nComments, PostComments], ignore_index=True)
            
  
           
        except:
            print('抓取留言出現錯誤喔傻屌A_A')
    except:
        print('這篇留言過少所以不用展開喔傻屌A_A')
    gc.collect()
    print('完成時間: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + '\n\n------------------')

正在抓: https://www.facebook.com/SourSoul.cc/posts/2709264255802475
fuckyou
完成!
完成時間: 2019-11-07 11:56:40

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2751125854949648
展開夠多留言： 0 次
完成!
完成時間: 2019-11-07 11:57:05

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2748409881887912
展開夠多留言： 0 次
加載： 1 次
完成!
完成時間: 2019-11-07 11:57:37

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2746738542055046
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
完成!
完成時間: 2019-11-07 11:58:20

------------------
正在抓: https://www.facebook.com/SourSoul.cc/photos/a.1979984188730489/2747892028606364/
展開夠多留言： 0 次
加載： 1 次
Click See More ： 1 times.
完成!
完成時間: 2019-11-07 11:58:57

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2743330989062468
完成!
完成時間: 2019-11-07 11:59:16

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2741417019253865
展開夠多留言： 0 次
完成!
完成時間: 2019-11-07 11:59:40

------------------
正在抓: https://www.facebook.com/SourSoul.c

展開夠多留言： 85 次
展開夠多留言： 86 次
展開夠多留言： 87 次
展開夠多留言： 88 次
展開夠多留言： 89 次
展開夠多留言： 90 次
展開夠多留言： 91 次
展開夠多留言： 92 次
展開夠多留言： 93 次
展開夠多留言： 94 次
展開夠多留言： 95 次
展開夠多留言： 96 次
展開夠多留言： 97 次
展開夠多留言： 98 次
展開夠多留言： 99 次
展開夠多留言： 100 次
展開夠多留言： 101 次
展開夠多留言： 102 次
展開夠多留言： 103 次
展開夠多留言： 104 次
展開夠多留言： 105 次
展開夠多留言： 106 次
展開夠多留言： 107 次
展開夠多留言： 108 次
展開夠多留言： 109 次
展開夠多留言： 110 次
展開夠多留言： 111 次
展開夠多留言： 112 次
展開夠多留言： 113 次
展開夠多留言： 114 次
展開夠多留言： 115 次
展開夠多留言： 116 次
展開夠多留言： 117 次
展開夠多留言： 118 次
展開夠多留言： 119 次
展開夠多留言： 120 次
展開夠多留言： 121 次
展開夠多留言： 122 次
展開夠多留言： 123 次
展開夠多留言： 124 次
展開夠多留言： 125 次
展開夠多留言： 126 次
展開夠多留言： 127 次
展開夠多留言： 128 次
展開夠多留言： 129 次
展開夠多留言： 130 次
展開夠多留言： 131 次
展開夠多留言： 132 次
展開夠多留言： 133 次
展開夠多留言： 134 次
展開夠多留言： 135 次
展開夠多留言： 136 次
展開夠多留言： 137 次
展開夠多留言： 138 次
展開夠多留言： 139 次
展開夠多留言： 140 次
展開夠多留言： 141 次
展開夠多留言： 142 次
展開夠多留言： 143 次
展開夠多留言： 144 次
展開夠多留言： 145 次
展開夠多留言： 146 次
展開夠多留言： 147 次
展開夠多留言： 148 次
展開夠多留言： 149 次
展開夠多留言： 150 次
展開夠多留言： 151 次
展開夠多留言： 152 次
展開夠多留言： 153 次
展開夠多留言： 154 次
展開夠多留言： 155 次
展開夠多留言： 156 次
展開夠多留言：

展開夠多留言： 0 次
完成!
完成時間: 2019-11-07 12:21:59

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2590006327728269
展開夠多留言： 0 次
完成!
完成時間: 2019-11-07 12:22:10

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2588315304564038
展開夠多留言： 0 次
完成!
完成時間: 2019-11-07 12:22:20

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2584234108305491
展開夠多留言： 0 次
Click See More ： 1 times.
完成!
完成時間: 2019-11-07 12:22:32

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2586162768112625
展開夠多留言： 0 次
加載： 1 次
完成!
完成時間: 2019-11-07 12:22:44

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2584258834969685
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次
加載： 5 次
加載： 6 次
加載： 7 次
加載： 8 次
加載： 9 次
加載： 10 次
加載： 11 次
加載： 12 次
加載： 13 次
加載： 14 次
加載： 15 次
加載： 16 次
加載： 17 次
加載： 18 次
加載： 19 次
加載： 20 次
加載： 21 次
加載： 22 次
加載： 23 次
加載： 24 次
加載： 25 次
加載： 26 次
加載： 27 次
加載： 28 次
加載： 29 次
加載： 30 次
加載： 31 次
加載： 32 次
加載： 33 次
加載： 34 次
加載： 35 次
加載： 36 次
加載： 

加載： 5 次
加載： 6 次
加載： 7 次
加載： 8 次
加載： 9 次
加載： 10 次
加載： 11 次
加載： 12 次
完成!
完成時間: 2019-11-07 12:32:10

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2537255333003369
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次
完成!
完成時間: 2019-11-07 12:32:24

------------------
正在抓: https://www.facebook.com/SourSoul.cc/photos/a.900923439969908/2535779016484334/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次
完成!
完成時間: 2019-11-07 12:32:38

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2532936546768581
展開夠多留言： 0 次
完成!
完成時間: 2019-11-07 12:32:49

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2532128266849409
展開夠多留言： 0 次
完成!
完成時間: 2019-11-07 12:33:00

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2529880960407473
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
完成!
完成時間: 2019-11-07 12:33:12

------------------
正在抓: https://www.facebook.com/SourSoul.cc/photos/a.900923439969908/2530274703701432/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
完成!
完成時間: 2019-11-07 12:33:24

----

展開夠多留言： 0 次
加載： 1 次
完成!
完成時間: 2019-11-07 12:38:55

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2506153802780189
展開夠多留言： 0 次
加載： 1 次
完成!
完成時間: 2019-11-07 12:39:06

------------------
正在抓: https://www.facebook.com/SourSoul.cc/photos/a.900923439969908/2506733596055543/
展開夠多留言： 0 次
完成!
完成時間: 2019-11-07 12:39:17

------------------
正在抓: https://www.facebook.com/SourSoul.cc/photos/a.900923439969908/2505227709539465/
展開夠多留言： 0 次
加載： 1 次
完成!
完成時間: 2019-11-07 12:39:29

------------------
正在抓: https://www.facebook.com/SourSoul.cc/photos/a.900923439969908/2504752586253644/
展開夠多留言： 0 次
加載： 1 次
完成!
完成時間: 2019-11-07 12:39:40

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2504574832938086
展開夠多留言： 0 次
加載： 1 次
完成!
完成時間: 2019-11-07 12:39:51

------------------
正在抓: https://www.facebook.com/SourSoul.cc/photos/a.900923439969908/2504448872950682/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
完成!
完成時間: 2019-11-07 12:40:04

------------------
正在抓: https://www.facebook.com/So

展開夠多留言： 0 次
完成!
完成時間: 2019-11-07 12:49:53

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2446089828786587
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次
加載： 5 次
完成!
完成時間: 2019-11-07 12:50:08

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2443852859010284
展開夠多留言： 0 次
完成!
完成時間: 2019-11-07 12:50:18

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2442849915777245
展開夠多留言： 0 次
完成!
完成時間: 2019-11-07 12:50:29

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2441787819216788
展開夠多留言： 0 次
完成!
完成時間: 2019-11-07 12:50:40

------------------
正在抓: https://www.facebook.com/SourSoul.cc/photos/a.900923439969908/2441992002529703/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次
完成!
完成時間: 2019-11-07 12:50:54

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2441962739199296
展開夠多留言： 0 次
加載： 1 次
完成!
完成時間: 2019-11-07 12:51:06

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2438243689571201
展開夠多留言： 0 次

完成!
完成時間: 2019-11-07 13:00:47

------------------
正在抓: https://www.facebook.com/SourSoul.cc/photos/a.1979984188730489/2356914104370827/
展開夠多留言： 0 次
完成!
完成時間: 2019-11-07 13:00:57

------------------
正在抓: https://www.facebook.com/SourSoul.cc/photos/a.833795846682668/2355958501133054/
完成!
完成時間: 2019-11-07 13:01:05

------------------
正在抓: https://www.facebook.com/SourSoul.cc/photos/a.1979984188730489/2355176924544545/
展開夠多留言： 0 次
完成!
完成時間: 2019-11-07 13:01:15

------------------
正在抓: https://www.facebook.com/SourSoul.cc/photos/a.1979984188730489/2353503058045265/
展開夠多留言： 0 次
完成!
完成時間: 2019-11-07 13:01:26

------------------
正在抓: https://www.facebook.com/SourSoul.cc/photos/a.1979984188730489/2352383901490514/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
完成!
完成時間: 2019-11-07 13:01:38

------------------
正在抓: https://www.facebook.com/SourSoul.cc/photos/a.1979984188730489/2350105668385004/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
完成!
完成時間: 2019-11-07 13:01:50

------------------
正在抓: https://www.facebook.com/SourSoul.cc/p

展開夠多留言： 0 次
加載： 1 次
完成!
完成時間: 2019-11-07 13:10:41

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2172913269437579
完成!
完成時間: 2019-11-07 13:10:49

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2171969756198597
完成!
完成時間: 2019-11-07 13:10:56

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2170742996321273
完成!
完成時間: 2019-11-07 13:11:03

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2170642806331292
完成!
完成時間: 2019-11-07 13:11:12

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2167393543322885
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
完成!
完成時間: 2019-11-07 13:11:24

------------------
正在抓: https://www.facebook.com/SourSoul.cc/photos/a.833795846682668/2166538943408345/
完成!
完成時間: 2019-11-07 13:11:32

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2166153190113587
加載： 1 次
完成!
完成時間: 2019-11-07 13:11:42

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2165969583465281
完成!


完成!
完成時間: 2019-11-07 13:20:19

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2006796989382542
完成!
完成時間: 2019-11-07 13:20:26

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/2002335259828715
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次
加載： 5 次
加載： 6 次
加載： 7 次
Click See More ： 1 times.
完成!
完成時間: 2019-11-07 13:20:42

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/1994780660584175
加載： 1 次
加載： 2 次
Click See More ： 1 times.
完成!
完成時間: 2019-11-07 13:20:53

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/1994461403949434
完成!
完成時間: 2019-11-07 13:21:01

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/1993973453998229
完成!
完成時間: 2019-11-07 13:21:08

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/1993932570668984
完成!
完成時間: 2019-11-07 13:21:18

------------------
正在抓: https://www.facebook.com/SourSoul.cc/posts/1988219847906923
完成!
完成時間: 2019-11-07 13:21:28

------------------
正在抓: http

# <font color=#8B4513 size=100 face="標楷體"> 合併 </font>

In [17]:
fkk = pd.merge(nPost,nComments, on='Link', how='outer', indicator='indicator_column',suffixes=['_文章','_留言'])
終於=fkk.loc[:, ['Content_文章','Time_文章', 'Comments','Content_留言','Link']]
終於

,Content_文章,Time_文章,Comments,Content_留言,Link
0,Bankee這次對新戶超好，國內最高竟然有20%回饋！還沒上車的一定要趕快跟上卡板邦的好康團...,2019-10-17 17:30,7.0,,https://www.facebook.com/SourSoul.cc/posts/270...
1,雖然開戶時碰到小插曲，但第四張東奧卡順利到手了！ PS 內附一張四喜風火輪，歡迎參觀！,2019-11-05 06:10,10.0,"期待永豐的東奧卡,我也期待！,讚讚！,東奧卡是實體戶嗎？要如何同時擁有「東奧卡與數位帳戶」。...",https://www.facebook.com/SourSoul.cc/posts/275...
2,隨機返現100%的台新街口聯名卡！你今天開刷了嗎？ 🤩Bankee最高20%：http://...,2019-11-04 06:00,7.0,"今天開刷。核卡就能綁定。好方便,不是被玩壞了嗎,其實只是把手機行拿掉保底15%，但活動還是照...",https://www.facebook.com/SourSoul.cc/posts/274...
3,同時有玉山 #世界卡 和 #Only卡 ，那真的可以同時累積高倍紅利，又享有機場接送和貴賓室...,2019-11-04 04:00,9.0,"謝謝分享另外想請教only卡的旅遊平安險是適用於哪個等級呢？謝謝,施進財,施進財 是商務卡等...",https://www.facebook.com/SourSoul.cc/posts/274...
4,【曬卡面】新光簽帳金融卡(東京奧運卡面) 順便一起拿了LINE Pay一卡通綁定禮！ 🤩Ba...,2019-11-03 21:00,9.0,"滿好看的也，要不要來去辦呢？,好快,沒想到USB竟然還有！,羨慕。我的提款卡面超醜😭問一個弱...",https://www.facebook.com/SourSoul.cc/photos/a....
5,聯邦和Visa攜手送好禮！ 感謝 輝哥卡省錢 分享！ 🤩Bankee最高20%：http:/...,2019-11-01 22:00,2.0,,https://www.facebook.com/SourSoul.cc/posts/274...
6,【0.7 Wallet】本站出貨進度公告 本站的早鳥團(1~5團)卡夾已全數到貨，然而官方 ...,2019-10-31 23:10,9.0,"名單已補上囉～大力感謝 SourSoul 信用卡快爆 揪團＆協助出貨☺️,辛苦啦！,辛苦您了...",https://www.facebook.com/SourSoul.cc/posts/274...
7,今天卡了一片空白，終於進去後跟我說額滿...我無言 🤩Bankee最高20%：http://...,2019-10-31 18:16,22.0,"好可惜,残念です,太累了不想搶了，搶到又得要強迫消費~,我上個月沒搶到，但是剛剛9:03搶到...",https://www.facebook.com/SourSoul.cc/posts/274...
8,【曬卡面】台新街口聯名卡（在街口支付裡面的樣子） 我比較喜歡咖咖的說⋯⋯街口支付 之後有考慮...,2019-10-30 04:00,12.0,"我也綁定好了 這次不知道是否可以MJ兩卡同時擁有,好快,收到了🎉,我比較想要張鈞甯的圖案.....",https://www.facebook.com/SourSoul.cc/photos/a....
9,There's No Text Content!,2019-10-28 07:16,5.0,"美美滴！,版大可以當推廣南韓大使了（你説，最近你去了幾次？）（一直放圖「毒」，害我好想去）",https://www.facebook.com/SourSoul.cc/photos/a....


# <font color=#8B4513 size=100 face="標楷體"> 保存資料</font>

In [18]:
終於.to_excel('C:\\Users\\Big data\\Desktop\\class\\web_crawler\\FB\\FB粉專SourSoul 信用卡快爆.xls')

# <font color=#8B4513 size=100 face="標楷體"> 資源參考</font>

In [666]:
#合併
https://morvanzhou.github.io/tutorials/data-manipulation/np-pd/3-7-pd-merge/
http://justimchung.blogspot.com/2018/06/pandas-dataframe.html

SyntaxError: invalid syntax (<ipython-input-666-1eb1a4973449>, line 2)

In [481]:
留言的留言

<div class="_5pcr userContentWrapper" data-ft='{"tn":"-R"}' style=""><div class="_1dwg _1w_m _q7o"><div class="_4r_y"><div class="_1wbl"></div></div><div><span class="_47we _42b7"></span><div class="l_c3pyo2v0u i_c3pynyi2f clearfix"><div class="clearfix y_c3pyo2ta3"><a aria-hidden="true" class="_5pb8 o_c3pynyi2g _8o _8s lfloat _ohe" data-ft='{"tn":"\u003C"}' href="https://www.facebook.com/OldFoxCreditCardDiary/?ref=nf&amp;hc_ref=ARTbuAU7B-eVAVBYoeX2j9OZAcOGbCBpVNZc58F4gtJufhbMg7rby_i_PuvgiOZ9gTM&amp;__xts__%5B0%5D=68.ARBq2P75I9mveCVW29_t78yFVhnzOSIY1PzAXQTmZOwXa-6AKJ2-bNpZk1V7BfwQI970cia4DrqWaI43GNQ2JdWso3S0ZQR2XW3B4Pe1o-5OiW_wY2U5SBZ4g55IeT1sHApaXJmzPUCFM1EuaOOZrmtS2lDclCUp8xbDXhhBrAQzyNnAkOrUeCJw2_4Teq-l_ThShJBEmyo5QigEJe2nwFpQzqORGs2lLkJwj2TdD35nyd8cc4qXpIvpUNV2D-J04k1IvN9GD4ZUzsq10wJWwWKBWIWMfPE_WW3fExNNxaru9lViq11VqmOxvT_85VSxdox7TtvuQ4TYBtHl7cS-K0w&amp;__tn__=%3C-R" tabindex="-1" target=""><div class="_38vo"><!-- react-mount-point-unstable --><div><img alt="" aria-label="老狐狸的信用卡日

# <font color=#8B4513 size=100 face="標楷體"> 保存資料</font>
